In [1]:
# Change directory to VSCode workspace root so that relative path loads work correctly. Turn this addition off with the DataScience.changeDirOnImportExport setting
# ms-python.python added
import os
try:
	os.chdir(os.path.join(os.getcwd(), '../../python-tools'))
	print(os.getcwd())
except:
	pass


  ### 1. Timing

  * How long does it take for a function to run?

  ```
  * The inner function (“wrapper”) will run the original function
  * But it’ll keep track of the time before and after doing so.
  * Before returning the result to the user, write the timing information to a logfile
  ```

In [2]:
import time
import functools



In [3]:
def logtime(func):

    def wrapper(*args, **kwargs):

        start_time = time.time()
        result = func(*args, **kwargs)
        total_time = time.time() - start_time

        with open('timelog.txt', 'a') as outfile:
            outfile.write(f'{time.time()}\t{func.__name__}\t{total_time}\n')

        return result

    return wrapper




In [4]:
@logtime

def slow_add(a, b):
    time.sleep(2)
    return a + b




In [5]:
@logtime

def slow_mul(a, b):
    time.sleep(3)
    return a * b




In [6]:
# test function
slow_add(100,100)


200

  records from timelogs:

  1565464610.880044	slow_add	2.0048961639404297


In [7]:
# alternative method
def timethis(func):
    '''
    decortor that reports the execution time.
    '''
    @functools.wraps(func)  
    def wrapper(*args, **kwargs):
        start = time.time()
        result = func(*args, **kwargs)
        end = time.time()
        print(func.__name__, end-start)
        return result
    return wrapper



In [8]:
@timethis
def countdown(n):
    '''
    counts down
    '''
    while n > 0:
        n -= 1



In [9]:
countdown(10000)


countdown 0.0009968280792236328


  ### 2. Run function once per minute

  * Raise an exception if we try to run a function more than once in 60 seconds

In [10]:
def once_per_minute(func):

    last_invoked = 0

    def wrapper(*args, **kwargs):

        nonlocal last_invoked

        elapsed_time = time.time() - last_invoked

        if elapsed_time < 60:

            raise RuntimeError(f"Only {elapsed_time} has passed")

        last_invoked = time.time()

        return func(*args, **kwargs)

    return wrapper




In [11]:
@once_per_minute

def add(a, b):
    return a + b




In [12]:
add(2,3)



5

In [13]:
#  add(2,3)

#  ---------------------------------------------------------------------------
#  RuntimeError                              Traceback (most recent call last)
#   in
#  ----> 1 add(2,3)
#   in wrapper(*args, **kwargs)
#       11         if elapsed_time < 60:
#       12
#  ---> 13             raise RuntimeError(f"Only {elapsed_time} has passed")
#       14
#       15         last_invoked = time.time()
#  RuntimeError: Only 10.496497869491577 has passed


  ### 3. Perform function per N seconds
  * Raise an exception if we try to run a function more than once in n seconds

In [14]:
def once_per_n(n):

    def middle(func):
        last_invoked = 0

        def wrapper(*args, **kwargs):
            nonlocal last_invoked
            elapsed_time = time.time() - last_invoked
            if elapsed_time < n:
                raise RuntimeError(f"Only {elapsed_time} has passed")

            last_invoked = time.time()
            return func(*args, **kwargs)

        return wrapper

    return middle




In [15]:
@once_per_n(10)

def add(a, b):
    return a + b




In [16]:
add(2,3)



5

In [17]:
#  add(2,3)

#  ---------------------------------------------------------------------------
#  RuntimeError                              Traceback (most recent call last)
#   in
#  ----> 1 add(2,3)
#   in wrapper(*args, **kwargs)
#        8             elapsed_time = time.time() - last_invoked
#        9             if elapsed_time < n:
#  ---> 10                 raise RuntimeError(f"Only {elapsed_time} has passed")
#       11
#       12             last_invoked = time.time()
#  RuntimeError: Only 0.025335073471069336 has passed
#  

  ### 4. Memoization
  * Cache the results of function calls, so we don’t need to call them again

In [18]:
import pickle




In [19]:
# Executes each time the decorated function is executed
def memoize(func):
    cache = {}

    # Executes each time the decorated function is executed
    def wrapper(*args, **kwargs):
        t = (pickle.dumps(args), pickle.dumps(kwargs))
        if t not in cache:
            print(f"Caching NEW value for {func.__name__}{args}")
            cache[t] = func(*args, **kwargs)
        else:
            print(f"Using OLD value for {func.__name__}{args}")

        return cache[t]

    return wrapper




In [20]:
@memoize
def add(a, b):
    print("Running add!")
    return a + b




In [21]:
@memoize
def mul(a, b):
    print("Running mul!")
    return a * b




In [22]:
add(2,2)




Caching NEW value for add(2, 2)
Running add!


4

In [23]:
add(2,2)


Using OLD value for add(2, 2)


4

  ### 5. Attributes
  * Give many objects the same attributes, but without using inheritance

In [24]:
def fancy_repr(self):
    return f"I'm a {type(self)}, with vars {vars(self)}"




In [25]:
def better_repr(c):
    c.__repr__ = fancy_repr
    def wrapper(*args, **kwargs):
        o = c(*args, **kwargs)
        return o
    return wrapper




In [26]:
# Alternative approach
def better_repr(c): #The decorated class
    c.__repr__ = fancy_repr
    return c #Return a callable — class




In [27]:
@better_repr
class Foo():
    def __init__(self, x, y):
        self.x = x
        self.y = y




In [28]:
f = Foo(10, [10, 20, 30])
print(f)


I'm a <class '__main__.Foo'>, with vars {'x': 10, 'y': [10, 20, 30]}


  * The **@object_birthday** decorator, when applied to a class, will add a new **created_at** attribute to new objects
  * Give every object its own birthday. This will contain the timestamp at which each instance was created

In [29]:
def object_birthday(c): #The decorated class
    def wrapper(*args, **kwargs):
        o = c(*args, **kwargs)
        o._created_at = time.time()
        return o
    return wrapper




In [30]:
@object_birthday
class Foo():
    def __init__(self, x, y):
        self.x = x
        self.y = y




In [31]:
f = Foo(10, [10, 20, 30])
print(f)
print(f._created_at)




1570225832.0968118


In [32]:
def object_birthday(c):
    c.__repr__ = fancy_repr  #Add a method to the class

    def wrapper(*args, **kwargs):
        o = c(*args, **kwargs)
        o._created_at = time.time()  #Add an attribute to the instance
        return o
    return wrapper




In [33]:
@object_birthday
class Foo():
    def __init__(self, x, y):
        self.x = x
        self.y = y





In [34]:
f = Foo(10, [10, 20, 30])
print(f)



I'm a <class '__main__.Foo'>, with vars {'x': 10, 'y': [10, 20, 30], '_created_at': 1570225832.158646}


 ### 6. Unwrapping a decorator
 * gain access to the original function by accessing the `__wrapped__` attribute.

In [35]:
# decorator has been implemented properly using @wraps
@timethis
def add(x,y):
    return x+y

original_add = add.__wrapped__
original_add(3,4)



7

 ### 7. Preserving function metadata
 * apply the @wraps decorator from functool library to the underlying wrapper function.

In [36]:
def timethis(func):
    '''
    decorator that reports the execution time.
    '''
    @functools.wraps(func)
    def wrapper(*args, **kwargs):
        start = time.time()
        result = func(*args, **kwargs)
        end = time.time()
        print(func.__name__, end-start)
        return result
    return wrapper


In [37]:
@timethis
def countdown(n:int):
    '''
    counts down
    '''
    while n>0:
        n-=1


In [38]:
countdown(100000)

countdown 0.006983280181884766


In [39]:
countdown.__name__

'countdown'

In [40]:
countdown.__doc__


'\n    counts down\n    '

In [41]:
countdown.__annotations__


{'n': int}

 ### 8. decorator that takes arguments

In [42]:
import logging

def logged(level, name = None, message = None):
    '''
    add logging to a function.
    level is the logging level, name is the logger name, and message is the log message. If name and message aren't specified, they default to the function's module and name
    '''
    def decorate(func):
        logname = name if name else func.__name__
        log = logging.getLogger(logname)
        logmsg = message if message else func.__name__

        @functools.wraps(func)
        def wrapper(*args, **kwargs):
            log.log(level, logmsg)
            return func(*args, **kwargs)
        return wrapper
    return decorate

In [43]:
# example usage
@logged(logging.DEBUG)
def add(x, y):
    return x+y


In [44]:
@logged(logging.CRITICAL)
def spam():
    print('Spam')


 ### 8. Type checking on function
 * use inspect.signature() function

In [45]:
import inspect


In [46]:
def typeassert(*ty_args, **ty_kwargs):
    def decorate(func):
        # if in optimized mode, disable type checking
        if not __debug__:
            return func
        
        # map function argument names to supplied types
        sig = inspect.signature(func)
        bound_types = sig.bind_partial(*ty_args, **ty_kwargs).arguments

        @functools.wraps(func)
        def wrapper(*args, **kwargs):
            bound_values = sig.bind(*args, **kwargs)
            # enforce type assertions across supplied arguments
            for name, value in bound_values.arguments.items():
                if name in bound_types:
                    if not isinstance(value, bound_types[name]):
                        raise TypeError(
                    'Argument {} must be {}'.format(name, bound_types[name]))
            
            return func(*args, **kwargs)
        return wrapper
    return decorate




In [47]:
@typeassert(int, z=int)
def spam(x, y, z=42):
    print(x,y,z)



In [48]:
spam(1,2,3)


1 2 3


In [49]:
spam(1, 'hello', 3)


1 hello 3


In [50]:
# spam(1,'hello','world')
# ---------------------------------------------------------------------------
# TypeError                                 Traceback (most recent call last)
#  in ()
# ----> 1 spam(1,'hello','world')

#  in wrapper(*args, **kwargs)
#      17                     if not isinstance(value, bound_types[name]):
#      18                         raise TypeError(
# ---> 19                     'Argument {} must be {}'.format(name, bound_types[name]))
#      20 
#      21             return func(*args, **kwargs)

# TypeError: Argument z must be 

In [51]:
def spam(x,y,z=42):
    pass

In [52]:
sig = inspect.signature(spam)
print(sig)

(x, y, z=42)


In [53]:
sig.parameters


mappingproxy({'x': <Parameter "x">,
              'y': <Parameter "y">,
              'z': <Parameter "z=42">})

In [54]:
sig.parameters['z']._default


42

In [55]:
sig.parameters['z'].kind



<_ParameterKind.POSITIONAL_OR_KEYWORD: 1>

In [56]:
bound_types = sig.bind_partial(int, z = int)
bound_types


<BoundArguments (x=<class 'int'>, z=<class 'int'>)>

In [57]:
bound_types.arguments


OrderedDict([('x', int), ('z', int)])

In [58]:
bound_values = sig.bind(1,2,3)
bound_values.arguments

OrderedDict([('x', 1), ('y', 2), ('z', 3)])

 ### 9. Decorator as part of a class
 * define decorator inside a class definition and apply it to other functions or methods

In [59]:
class A:
    #decorator as an instance method
    def decorator1(self, func):
        @functools.wraps(func)
        def wrapper(*args, **kwargs):
            print('Decorator 1')
            return func(*args, **kwargs)
        return wrapper

    # decorator as a class method
    @classmethod
    def decorator2(cls, func):
        @functools.wraps(func)
        def wrapper(*args, **kwargs):
            print('Decorator 2')
            return func(*args, **kwargs)
        return wrapper



In [60]:
# example of how the two decorators would be applied
# as an instance method
a = A()
@a.decorator1
def spam():
    pass

# as a class method
@A.decorator2
def grok():
    pass


In [61]:
# build-in @property decorator is a class with getter(), setter() and deleter() methods that each act as a decorator

class Person:
    #create a propety instance
    first_name = property()

    #apply decorator methods
    @first_name.getter
    def first_name(self):
        return self._first_name

    @first_name.setter
    def first_name(self, value):
        if not isinstance(self, value):
            raise TypeError('Expected a string')
        self._first_name = value

    


 ### 10. Decorator as classes
 * Decorator works both inside and boutside class definitions, it implements the `__call__()` and `__get__()` methods.

In [62]:
import types


In [63]:
class Profiled:
    '''
    defines a class that puts a simple profiling layer around another function
    '''

    def __init__(self, func):
        functools.wraps(func)(self)
        self.ncalls = 0

    def __call__(self, *args, **kwargs):
        self.ncalls +=1
        return self.__wrapped__(*args, **kwargs)

    def __get__(self, instance, cls):
        if instance is None:
            return self
        else:
            return types.MethodType(self, instance)


In [64]:
#use as a normal decorator , either inside or outside of a class
@Profiled
def add(x,y):
    return x+y

class Spam:
    @Profiled
    def bar(self, x):
        print(self, x)


In [65]:
add(2,3)


5

In [66]:
add(4,5)


9

In [67]:
add.ncalls


2

In [68]:
s = Spam()


In [69]:
s.bar(1)


<__main__.Spam object at 0x00000259414F87B8> 1


In [70]:
s.bar(2)


<__main__.Spam object at 0x00000259414F87B8> 2


In [71]:
s.bar(3)


<__main__.Spam object at 0x00000259414F87B8> 3


In [72]:
Spam.bar.ncalls


3

In [73]:
# alternative formulation of the decorator using closures and nonlocal variables

def profiled(func):
    ncalls = 0
    @functools.wraps(func)
    def wrapper(*args, **kwargs):
        nonlocal ncalls
        ncalls += 1
        return func(*args, **kwargs)
    wrapper.ncalls = lambda: ncalls
    return wrapper



In [74]:
# example 
@profiled
def add(x,y):
    return x+y


In [75]:
add(2,3)


5

In [76]:
add(4,5)


9

In [77]:
add.ncalls()


2

 ### 11. Applying decorator to class and static methods
 * Make sure that the decorators are applied before `@classmethod` or `@staticmethod`.

In [78]:
# a simple decorator
def timethis(func):
    @functools.wraps(func)
    def wrapper(*args, **kwargs):
        start = time.time()
        result = func(*args, **kwargs)
        end = time.time()
        print(end - start)
        return result
    return wrapper



In [79]:
# class sllustrating application of the decorator to different kinds of methods 

class Spam:
    @timethis
    def instance_method(self, n):
        print(self, n)
        while n>0:
            n-=1

    @classmethod
    @timethis
    def class_method(cls, n):
        print(cls,n)
        while n>0:
            n-=1

    @staticmethod
    @timethis
    def static_method(n):
        print(n)
        while n>0:
            n-=1


In [80]:
s = Spam()
s.instance_method(1000000)


<__main__.Spam object at 0x00000259414F82B0> 1000000
0.07080483436584473


In [81]:
Spam.class_method(1000000)

<class '__main__.Spam'> 1000000
0.08377575874328613


In [82]:
Spam.static_method(1000000)


1000000
0.07380080223083496


 ### 12. Decorators that add arguments to wrapped functions
 * extra arguments can be injected into the calling signature using keyword-only arguments.

In [83]:
def optional_debug(func):
    @functools.wraps(func)
    def wrapper(*args, debug = False, **kwargs):
        if debug:
            print('calling', func.__name__)
        return func(*args, **kwargs)
    return wrapper

In [84]:
@optional_debug
def spam(a,b,c):
    print(a,b,c)


In [85]:
spam(1,2,3)


1 2 3


In [86]:
spam(1,2,3, debug = True)


calling spam
1 2 3


In [87]:
# if the @optional_debug decorator was applied to a functin that already had a debug argument.
def optional_debug(func):
    if 'debug' in inspect.getargspec(func).args:
        raise TypeError('debug argument already defined')
    @functools.wraps(func)
    def wrapper(*args, debug=False, **kwargs):
        if debug:
            print('calling', func.__name__)
        return func(*args, **kwargs)

    sig = inspect.signature(func)
    parameters = list(sig.parameters.values())
    parameters.append(inspect.Parameter('debug', 
                                        inspect.Parameter.KEYWORD_ONLY,default=False))
    wrapper.__signature__ = sig.replace(parameters=parameters)
    return wrapper


In [88]:
@optional_debug
def add(x,y):
    return x+y


C:\Users\Nan\AppData\Local\conda\conda\envs\ipykernel_py3\lib\site-packages\ipykernel_launcher.py:3: DeprecationWarning: inspect.getargspec() is deprecated since Python 3.0, use inspect.signature() or inspect.getfullargspec()
  This is separate from the ipykernel package so we can avoid doing imports until


In [89]:
print(inspect.signature(add))


(x, y, *, debug=False)


 ### 13. Enforcing an argument signature
 * Use the `Signature` and `Parameter` classes from `inspect` module

In [90]:
# example of creating a function signature
# make a signature for a func(x, y=42,*,z=None)
parms = [inspect.Parameter('x', inspect.Parameter.POSITIONAL_OR_KEYWORD),
        inspect.Parameter('y', inspect.Parameter.POSITIONAL_OR_KEYWORD, default=42),
        inspect.Parameter('z', inspect.Parameter.KEYWORD_ONLY, default= None)]
sig = inspect.Signature(parms)
sig

<Signature (x, y=42, *, z=None)>

In [91]:
# once a signature object is created, it can be easily bound to *args and **kwargs using the signature's bind() method.

def func(*args, **kwargs):
    bound_values = sig.bind(*args, **kwargs)
    for name, value in bound_values.arguments.items():
        print(name, value)


In [92]:
func(1,2,z=3)


x 1
y 2
z 3


In [93]:
func(1)


x 1


In [94]:
func(1, z=3)


x 1
z 3


In [95]:
func(y=2,x =1)


x 1
y 2


In [96]:
# func(1,2,3,4)
# ----> 1 func(1,2,3,4)

#  in func(*args, **kwargs)
#       2 
#       3 def func(*args, **kwargs):
# ----> 4     bound_values = sig.bind(*args, **kwargs)
#       5     for name, value in bound_values.arguments.items():
#       6         print(name, value)

# ~\AppData\Local\conda\conda\envs\ipykernel_py3\lib\inspect.py in bind(*args, **kwargs)
#    2995         if the passed arguments can not be bound.
#    2996         """
# -> 2997         return args[0]._bind(args[1:], kwargs)
#    2998 
#    2999     def bind_partial(*args, **kwargs):

# ~\AppData\Local\conda\conda\envs\ipykernel_py3\lib\inspect.py in _bind(self, args, kwargs, partial)
#    2922                         # argument
#    2923                         raise TypeError(
# -> 2924                             'too many positional arguments') from None
#    2925 
#    2926                     if param.kind == _VAR_POSITIONAL:

# TypeError: too many positional arguments

In [97]:
# func(y=2)
# ---------------------------------------------------------------------------
# TypeError                                 Traceback (most recent call last)
#  in 
# ----> 1 func(y=2)

#  in func(*args, **kwargs)
#       2 
#       3 def func(*args, **kwargs):
# ----> 4     bound_values = sig.bind(*args, **kwargs)
#       5     for name, value in bound_values.arguments.items():
#       6         print(name, value)

# ~\AppData\Local\conda\conda\envs\ipykernel_py3\lib\inspect.py in bind(*args, **kwargs)
#    2995         if the passed arguments can not be bound.
#    2996         """
# -> 2997         return args[0]._bind(args[1:], kwargs)
#    2998 
#    2999     def bind_partial(*args, **kwargs):

# ~\AppData\Local\conda\conda\envs\ipykernel_py3\lib\inspect.py in _bind(self, args, kwargs, partial)
#    2910                             msg = 'missing a required argument: {arg!r}'
#    2911                             msg = msg.format(arg=param.name)
# -> 2912                             raise TypeError(msg) from None
#    2913             else:
#    2914                 # We have a positional argument to process

# TypeError: missing a required argument: 'x'

In [98]:
# argument duplication
# func(1, y=2, x=3)
# ---------------------------------------------------------------------------
# TypeError                                 Traceback (most recent call last)
#  in 
# ----> 1 func(1, y=2, x=3)

#  in func(*args, **kwargs)
#       2 
#       3 def func(*args, **kwargs):
# ----> 4     bound_values = sig.bind(*args, **kwargs)
#       5     for name, value in bound_values.arguments.items():
#       6         print(name, value)

# ~\AppData\Local\conda\conda\envs\ipykernel_py3\lib\inspect.py in bind(*args, **kwargs)
#    2995         if the passed arguments can not be bound.
#    2996         """
# -> 2997         return args[0]._bind(args[1:], kwargs)
#    2998 
#    2999     def bind_partial(*args, **kwargs):

# ~\AppData\Local\conda\conda\envs\ipykernel_py3\lib\inspect.py in _bind(self, args, kwargs, partial)
#    2936                         raise TypeError(
#    2937                             'multiple values for argument {arg!r}'.format(
# -> 2938                                 arg=param.name)) from None
#    2939 
#    2940                     arguments[param.name] = arg_val

# TypeError: multiple values for argument 'x'

In [99]:
# example of enforcing function signatures
# base class is defined an extremely general purpose version of __init__
# subclasses are expected to supply an expected signature
def make_sig(*names):
    parms = [inspect.Parameter(name, inspect.Parameter.POSITIONAL_OR_KEYWORD)           for name in names]
    return inspect.Signature(parms)

class Structure:
    __signature__ = make_sig()
    def __init__(self, *args, **kwargs):
        bound_values = self.__signature__.bind(*args, **kwargs)
        for name, value in bound_values.arguments.items():
            setattr(self, name, value)



In [100]:
# example useage
class Stock(Structure):
    __signature__ = make_sig('name', 'shares', 'price')

class Point(Structure):
    __signature__ = make_sig('x','y')


In [101]:
inspect.signature(Stock)


<Signature (name, shares, price)>

In [102]:
s1 = Stock('SBUX', 100, 85.92)



In [103]:
# s2 = Stock('SBUX', 100)


# s2 = Stock('SBUX', 100)
# ---------------------------------------------------------------------------
# TypeError                                 Traceback (most recent call last)
#  in 
# ----> 1 s2 = Stock('SBUX', 100)

#  in __init__(self, *args, **kwargs)
#       9     __signature__ = make_sig()
#      10     def __init__(self, *args, **kwargs):
# ---> 11         bound_values = self.__signature__.bind(*args, **kwargs)
#      12         for name, value in bound_values.arguments.items():
#      13             setattr(self, name, value)

# ~\AppData\Local\conda\conda\envs\ipykernel_py3\lib\inspect.py in bind(*args, **kwargs)
#    2995         if the passed arguments can not be bound.
#    2996         """
# -> 2997         return args[0]._bind(args[1:], kwargs)
#    2998 
#    2999     def bind_partial(*args, **kwargs):

# ~\AppData\Local\conda\conda\envs\ipykernel_py3\lib\inspect.py in _bind(self, args, kwargs, partial)
#    2910                             msg = 'missing a required argument: {arg!r}'
#    2911                             msg = msg.format(arg=param.name)
# -> 2912                             raise TypeError(msg) from None
#    2913             else:
#    2914                 # We have a positional argument to process

# TypeError: missing a required argument: 'price'

In [104]:
# s3 = Stock('SBUX', 100, 85.92, shares = 50)
# ---------------------------------------------------------------------------
# TypeError                                 Traceback (most recent call last)
#  in 
# ----> 1 s3 = Stock('SBUX', 100, 85.92, shares = 50)

#  in __init__(self, *args, **kwargs)
#       9     __signature__ = make_sig()
#      10     def __init__(self, *args, **kwargs):
# ---> 11         bound_values = self.__signature__.bind(*args, **kwargs)
#      12         for name, value in bound_values.arguments.items():
#      13             setattr(self, name, value)

# ~\AppData\Local\conda\conda\envs\ipykernel_py3\lib\inspect.py in bind(*args, **kwargs)
#    2995         if the passed arguments can not be bound.
#    2996         """
# -> 2997         return args[0]._bind(args[1:], kwargs)
#    2998 
#    2999     def bind_partial(*args, **kwargs):

# ~\AppData\Local\conda\conda\envs\ipykernel_py3\lib\inspect.py in _bind(self, args, kwargs, partial)
#    2936                         raise TypeError(
#    2937                             'multiple values for argument {arg!r}'.format(
# -> 2938                                 arg=param.name)) from None
#    2939 
#    2940                     arguments[param.name] = arg_val

# TypeError: multiple values for argument 'shares'